importando as bibliotecas


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd

# criar o navegador
nav = webdriver.Chrome()

In [1]:
# importar/visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [2]:
import time

definição de busca do google shopping


In [3]:

#
#produto = 'iphone 12 64gb'
#termos_banidos = 'mini watch'
#preco_min = 3000
#preco_max = 3500

##



In [12]:

##criando função para verificar se o produto encontrado tem termos banidos

def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    
    return tem_termos_banidos      
            
            
#########################################################################          
def verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produtos = False

    return tem_todos_termos_produtos 

#########################################################################
def busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max):
    
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_nome_produto = produto.split(' ')
    
    lista_ofertas = [] #tupla que vai receber os resultados da busca
    
    #entrar no google
    nav.get('https://www.google.com/')
    # pesquisando o produto
    nav.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea').send_keys(produto, Keys.ENTER) 

    #entrar na aba shopping
    time.sleep(5)
    
    elementos = nav.find_elements("class name", "hdtb-mitem")

    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    time.sleep(4)    

    #pegar as infos do produto
    lista_resultados = nav.find_elements('class name', 'sh-dgr__grid-result')

    for resultado in lista_resultados:
        preco = resultado.find_element('class name', 'a8Pemb').text
        nome = resultado.find_element('class name', 'tAxDx').text
        nome = nome.lower()

        ##
        #analisar se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)


        #analisar se ele tem TODOS os termos do nome do produto

        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)

        #selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produtos = True
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = resultado.find_element('class name', 'a8Pemb').text #tratando o valor(preco)
            preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')

            #tratando para remover caracteres indesejados como(2500 + impostos)
            indesejado ='+abcdefghijklmnopqrstuvwxyz'

            preco = ''.join([char for char in preco if char not in indesejado])
            preco = float(preco)

            preco_min = float(preco_min)
            preco_max = float(preco_max)

            #se o preco ta entre preco_min e preco_max
            if  preco_min <= preco <= preco_max:
                    elemento_referencia = resultado.find_element('class name', 'bONr3b')
                    elemento_pai = elemento_referencia.find_element('xpath', '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link)) 
    
    return lista_ofertas

#########################################################################


def busca_buscape(nav, produto, termos_banidos, preco_min, preco_max):
    
    #tratar os valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_nome_produto = produto.split(' ')
    preco_min = float(preco_min)
    preco_max = float(preco_max)
    
    lista_ofertas1 = [] #tupla que vai receber os resultados da busca
    
    
    #buscar no buscapé
    nav.get('https://www.buscape.com.br/')
    nav.find_element('xpath', '/html/body/div[1]/main/header/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    #pegar os resultados
    lista_resultados = nav.find_elements('class name', 'ProductCard_ProductCard_Inner__tsD4M')
    
    
    
    
    time.sleep(4)
        
        
        
        
    for resultado in lista_resultados:
        preco = resultado.find_element('class name', 'Text_MobileHeadingS__Zxam2').text
        nome = resultado.find_element('class name', 'ProductCard_ProductCard_Name__LT7hv').text
        nome = nome.lower()
        link = resultado.get_attribute('href')
        
        
        
        
        #print(nome, preco, link,  '\n')
        #analisar se o resultado tem termo banido e tem todos os termos do produto
        #analisar se ele não tem nenhum termo banido
        
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        #analisar se ele tem TODOS os termos do nome do produto

        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)
        #print(tem_termos_banidos, tem_todos_termos_produtos)
        #analisar se o preco esta entre preco min e preco max
        if not tem_termos_banidos and tem_todos_termos_produtos: ##se os dados batem continua---
            #tratando o preco
            preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
            #tratando para remover caracteres indesejados como(2500 + impostos)
            indesejado ='+abcdefghijklmnopqrstuvwxyz'

            preco = ''.join([char for char in preco if char not in indesejado])
            preco = float(preco)
            #print(preco, nome, link)
            if preco_min <= preco <= preco_max:
                lista_ofertas1.append((nome, preco, link))
    
    #retornar lista de ofertas
    return lista_ofertas
    

In [9]:
for linha in tabela_produtos.index:
    print(linha)


0
1


In [20]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    
    produto = tabela_produtos.loc[linha,'Nome']
    termos_banidos = tabela_produtos.loc[linha,'Termos banidos']
    preco_min = tabela_produtos.loc[linha,'Preço mínimo']
    preco_max = tabela_produtos.loc[linha,'Preço máximo']
    
    ###################



    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_google_shopping = None

    ###################



    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_buscape = None
        
    break
    ###################
display(tabela_ofertas)

,produto,preco,link
0,usado: iphone 12 64gb branco muito bom - troca...,3029.0,https://www.google.com/url?url=https://www.cas...
1,iphone 12 64gb azul - excelente,3041.1,https://www.google.com/url?url=https://www.tro...
2,usado: iphone 12 64gb branco excelente - troca...,3219.0,https://www.google.com/url?url=https://www.cas...
3,usado: iphone 12 64gb azul excelente - trocafone,3219.0,https://www.google.com/url?url=https://www.cas...


exportar para o excel

In [26]:
#exportar pro excel

tabela_ofertas.to_excel('Ofertas.xlsx', index=False, encoding='urls')

### EMAIL ###

In [27]:
import pathlib
import email.message
import mimetypes
import smtplib
from email.message import EmailMessage

caminho_tabela = pathlib.Path(r'Ofertas.xlsx')

if len(tabela_ofertas) > 0:


    corpo_email = f"""
            Prezados, bom dia
            
    Segue em anexo a tabela de ofertas dos produtos indicados.
    
    Qualquer dúvida estou à disposição.

    Att., João Paulo

            """



    msg = EmailMessage()
    msg['Subject'] = 'Tabela de Ofertas'
    msg['From'] = 'ztyxmad@gmail.com'
    msg['To'] = 'jdelbin1@gmail.com'
    password = 'tqqd wetk hdvs mkfg'
    msg.add_header('Content-Type', 'text/html')
    msg.set_content(corpo_email)


            # ANEXO
    import mimetypes
    attachment = pathlib.Path.cwd() / caminho_tabela 

    mime_type, encoding = mimetypes.guess_type(attachment.name)
    with open(attachment, 'rb') as fp:
        dados = fp.read()
    msg.add_attachment(dados, filename=f'Ofertas.xlsx', maintype='application', subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    
    #############


    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()

    s.login(msg['From'], password)
    s.send_message(msg)

    print('email enviado')

email enviado
